<a href="https://colab.research.google.com/github/RoboTuan/ML4IOT_HMW/blob/main/HMW2/HW2_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile


In [5]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--model', type=str, required=True, help='model name')
#parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
#parser.add_argument('--silence', action='store_true', help='add silence')
#args = parser.parse_args()

mfcc=False

In [6]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [7]:
zip_path = tf.keras.utils.get_file(
        origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
        fname='mini_speech_commands.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')
data_dir = os.path.join('.', 'data', 'mini_speech_commands')
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
total = 8000

182083584/182082353 [==============================] - 8s 0us/step


In [13]:
train =  tf. convert_to_tensor(pd.read_csv('https://raw.githubusercontent.com/RoboTuan/ML4IOT_HMW/main/HMW2/kws_train_split.txt', header=None).to_numpy())
validation =  tf. convert_to_tensor(pd.read_csv('https://raw.githubusercontent.com/RoboTuan/ML4IOT_HMW/main/HMW2/kws_val_split.txt', header=None).to_numpy())
test =  tf. convert_to_tensor(pd.read_csv('https://raw.githubusercontent.com/RoboTuan/ML4IOT_HMW/main/HMW2/kws_test_split.txt', header=None).to_numpy())

train

<tf.Tensor: shape=(6400, 1), dtype=string, numpy=
array([[b'./data/mini_speech_commands/down/baeac2ba_nohash_0.wav'],
       [b'./data/mini_speech_commands/up/f7b43563_nohash_1.wav'],
       [b'./data/mini_speech_commands/down/10ace7eb_nohash_3.wav'],
       ...,
       [b'./data/mini_speech_commands/stop/772db621_nohash_0.wav'],
       [b'./data/mini_speech_commands/yes/324210dd_nohash_1.wav'],
       [b'./data/mini_speech_commands/no/f17be97f_nohash_2.wav']],
      dtype=object)>

In [9]:
train_files = train
val_files = validation
test_files = test